#### Example Usage: Build Scaling Relation with the Traditional Method

##### Workflow

1. Prepare Adsorption Energy Data

2. Decide (Two) Descriptors and Group Member

3. Run the Relation Builder (and Check Metrics)

4. Visualize Results with Volcano Plots and More
